# Generating MIMO controller data
This series of notebooks explores the idea of emulating complex controller behavior with an LSTM. This first notebook is used to generate data from the TCLab for use in training the LSTM. The LSTM can then be used in place of the controller (or any other controller it is trained on), or used to evaluate the controller performance for anomaly detection.

Of particular significance for this series of notebooks is the fact that this controller is for a multiple input, multiple output (MIMO) system, and is therefore more complex. The model takes about 2 seconds to solve for each iteration, meaning the data is set to only come in 3 second intervals. This will be significant for showing an advantage of using an LSTM later on.

In [1]:
import tclab
import numpy as np
import time
import matplotlib.pyplot as plt
from gekko import GEKKO
import random
import pandas as pd


In [2]:
# Connect to Arduino
a = tclab.TCLab()
# Find current T1, T2
print('Temperature 1: {0:0.2f} °C'.format(a.T1))
print('Temperature 2: {0:0.2f} °C'.format(a.T2))

TCLab version 0.4.9
--- Serial Ports ---


RuntimeError: No Arduino device found.

In [ ]:
# Run time in minutes
run_time = 240.0

# Number of cycles with 3 second intervals
loops = int(20.0*run_time)
tm = np.zeros(loops)

# Temperature (K)
T1 = np.ones(loops) * a.T1 # temperature (degC)
Tsp1 = np.ones(loops) * a.T1 # set point (degC)
T2 = np.ones(loops) * a.T2 # temperature (degC)
Tsp2 = np.ones(loops) * a.T2 # set point (degC)

# Heater set point steps
end = 15 # leave 1st 15 seconds of Q1s as 0
while end <= loops:
    start = end
    end += random.randint(200,600) # keep new Q1s value for anywhere from 10 to 30 minutes
    Tsp1[start:end] = random.randint(40,50)
    
# Heater set point steps
end = 15 # leave 1st 15 seconds of Q1s as 0
while end <= loops:
    start = end
    end += random.randint(200,600) # keep new Q1s value for anywhere from 10 to 30 minutes
    Tsp2[start:end] = random.randint(40,50)

# heater values
Q1s = np.ones(loops) * 0.0
Q2s = np.ones(loops) * 0.0

plt.plot(Tsp1)
plt.plot(Tsp2)

In [ ]:
#########################################################
# Initialize Model
#########################################################
m = GEKKO(name='tclab-mpc',remote=False)

# 60 second time horizon, steps of 3 sec
m.time = np.linspace(0,60,21)

# Parameters
U = m.FV(value=2.55,name='u')
tau = m.FV(value=19.55,name='tau')
alpha1 = m.FV(value=0.0081)   # W / % heater
alpha2 = m.FV(value=0.0058) # W / % heater

# Manipulated variables
Q1 = m.MV(value=0,name='Q1')
Q1.STATUS = 1  # use to control temperature
Q1.FSTATUS = 0 # no feedback measurement
Q1.LOWER = 0.0
Q1.UPPER = 100.0
Q1.DMAX = 40.0
Q1.COST = 0.0
Q1.DCOST = 1.0

Q2 = m.MV(value=0,name='Q2')
Q2.STATUS = 1  # use to control temperature
Q2.FSTATUS = 0 # no feedback measurement
Q2.LOWER = 0.0
Q2.UPPER = 100.0
Q2.DMAX = 40.0 # 5-10
Q2.COST = 0.0
Q2.DCOST = 1.0 # increase

# Controlled variable
TC1 = m.CV(value=T1[0],name='TC1')
TC1.STATUS = 1     # minimize error with setpoint range
TC1.FSTATUS = 1    # receive measurement
TC1.TR_INIT = 1    # reference trajectory
TC1.TAU = 10       # time constant for response

# Controlled variable
TC2 = m.CV(value=T2[0],name='TC2')
TC2.STATUS = 1     # minimize error with setpoint range
TC2.FSTATUS = 1    # receive measurement
TC2.TR_INIT = 1    # reference trajectory
TC2.TAU = 10       # time constant for response

# State variables
TH1 = m.SV(value=T1[0])
TH2 = m.SV(value=T2[0])

Ta = m.Param(value=23.0+273.15)     # K
mass = m.Param(value=4.0/1000.0)    # kg
Cp = m.Param(value=0.5*1000.0)      # J/kg-K    
A = m.Param(value=10.0/100.0**2)    # Area not between heaters in m^2
As = m.Param(value=2.0/100.0**2)    # Area between heaters in m^2
eps = m.Param(value=0.9)            # Emissivity
sigma = m.Const(5.67e-8)            # Stefan-Boltzmann

# Heater temperatures
T1i = m.Intermediate(TH1+273.15)
T2i = m.Intermediate(TH2+273.15)

# Heat transfer between two heaters
Q_C12 = m.Intermediate(U*As*(T2i-T1i)) # Convective
Q_R12 = m.Intermediate(eps*sigma*As*(T2i**4-T1i**4)) # Radiative

# Semi-fundamental correlations (energy balances)
m.Equation(TH1.dt() == (1.0/(mass*Cp))*(U*A*(Ta-T1i) \
                    + eps * sigma * A * (Ta**4 - T1i**4) \
                    + Q_C12 + Q_R12 \
                    + alpha1*Q1))

m.Equation(TH2.dt() == (1.0/(mass*Cp))*(U*A*(Ta-T2i) \
                    + eps * sigma * A * (Ta**4 - T2i**4) \
                    - Q_C12 - Q_R12 \
                    + alpha2*Q2))

# Empirical correlations (lag equations to emulate conduction)
m.Equation(tau * TC1.dt() == -TC1 + TH1)
m.Equation(tau * TC2.dt() == -TC2 + TH2)

# Global Options
m.options.IMODE   = 6 # MPC
m.options.CV_TYPE = 1 # Objective type
m.options.NODES   = 3 # Collocation nodes
m.options.SOLVER  = 3 # 1=APOPT, 3=IPOPT
##################################################################

# Create plot
plt.figure()
plt.ion()
plt.show()

# Main Loop
start_time = time.time()
prev_time = start_time
try:
    for i in range(1,loops):
        # Sleep time
        sleep_max = 3.0
        sleep = sleep_max - (time.time() - prev_time)
        if sleep>=0.01:
            time.sleep(sleep)
        else:
            time.sleep(0.01)

        # Record time and change in time
        t = time.time()
        dt = t - prev_time
        prev_time = t
        tm[i] = t - start_time

        # Read temperatures in Kelvin 
        T1[i] = a.T1
        T2[i] = a.T2

        ###############################
        ### MPC CONTROLLER          ###
        ###############################
        TC1.MEAS = T1[i]
        TC2.MEAS = T2[i]
        # input setpoint with deadband +/- DT
        DT = 0.1
        TC1.SPHI = Tsp1[i] + DT
        TC1.SPLO = Tsp1[i] - DT
        TC2.SPHI = Tsp2[i] + DT
        TC2.SPLO = Tsp2[i] - DT
        # solve MPC
        try:
            m.solve(disp=False)   
        except:
            print('No Solution (t = %i)',tm[i])
        # test for successful solution
        if (m.options.APPSTATUS==1):
            # retrieve the first Q value
            Q1s[i] = Q1.NEWVAL
            Q2s[i] = Q2.NEWVAL
        else:
            # not successful, set heater to zero
            Q1s[i] = 0        
            Q2s[i] = 0        

        # Write output (0-100)
        a.Q1(Q1s[i])
        a.Q2(Q2s[i])

        # Plot
        plt.clf()
        ax=plt.subplot(3,1,1)
        ax.grid()
        plt.plot(tm[0:i],T1[0:i],'ro',MarkerSize=3,label=r'$T_1$')
        plt.plot(tm[0:i],Tsp1[0:i],'k-',LineWidth=2,label=r'$T_1 SP$')
        plt.ylabel('Temperature (degC)')
        plt.legend(loc='best')
        ax=plt.subplot(3,1,2)
        ax.grid()
        plt.plot(tm[0:i],T2[0:i],'ro',MarkerSize=3,label=r'$T_2$')
        plt.plot(tm[0:i],Tsp2[0:i],'g-',LineWidth=2,label=r'$T_2 SP$')
        plt.ylabel('Temperature (degC)')
        plt.legend(loc='best')
        ax=plt.subplot(3,1,3)
        ax.grid()
        plt.plot(tm[0:i],Q1s[0:i],'r-',LineWidth=3,label=r'$Q_1$')
        plt.plot(tm[0:i],Q2s[0:i],'b:',LineWidth=3,label=r'$Q_2$')
        plt.ylabel('Heaters')
        plt.xlabel('Time (sec)')
        plt.legend(loc='best')
        plt.draw()
        plt.pause(0.05)

    # Turn off heaters
    a.Q1(0)
    a.Q2(0)
    print('Shutting down') 

# Allow user to end loop with Ctrl-C           
except KeyboardInterrupt:
    # Disconnect from Arduino
    a.Q1(0)
    a.Q2(0)
    print('Shutting down')
    a.close()

# Make sure serial connection still closes when there's an error
except:           
    # Disconnect from Arduino
    a.Q1(0)
    a.Q2(0)
    print('Error: Shutting down')
    a.close()
    raise

In [ ]:
data = pd.DataFrame()
data['time'] = tm
data['Q1'] = Q1s
data['T1'] = T1
data['Tsp1'] = Tsp1
data['Q2'] = Q2s
data['T2'] = T2
data['Tsp2'] = Tsp2

data[['Tsp1','T1','Tsp2','T2']].plot()
filename = 'MIMO_'+str(int(run_time))+'min_data.csv'
data.to_csv(filename,index=False)

In [ ]:
# Interpolate data 

data['time'] = data['time'].astype(int)
data = data.set_index('time')
new_index = range(max(data.index)+1)
data = data.reindex(new_index,fill_value=np.nan)

# for i, row in data.iterrows():
#     print(i,row)

In [ ]:
linear_interp_cols = ['Q1','T1','Q2','T2']
pad_interp_cols = ['Tsp1','Tsp2']
data[linear_interp_cols] = data[linear_interp_cols].interpolate(method='linear')
data[pad_interp_cols] = data[pad_interp_cols].interpolate(method='pad')


data[['Tsp1','T1','Tsp2','T2']].plot()
filename = 'MIMO_'+str(int(run_time))+'min_data_interp.csv'
data.to_csv(filename,index=False)

In [ ]:
n = 100
data[['Tsp1','T1','Tsp2','T2']][n:n+300].plot()

In [ ]:
data[['Q1','Q2']][n:n+300].plot()

In [ ]:
# Save gekko model
import pickle

gekko_dict = {'model':m,
              'TC1':TC1,
              'TC2':TC2,
              'Q1':Q1,
              'Q2':Q2
             }
# gekko_dict['model'] = m

model_name = 'mpc_'+str(int(run_time))+'.pkl'
pickle.dump(gekko_dict, open(model_name, 'wb'))